In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame()

In [3]:
count = 50000

while(True):
    cur = pd.read_csv("../../data/global acc/result" + str(count) + ".csv", skipinitialspace=True, header=None)
    df = df.append(cur, ignore_index=True)
    if count == 500000:
        break
    count += 50000
cur = pd.read_csv("../../data/global acc/result_final.csv", skipinitialspace=True, header=None)
df = df.append(cur, ignore_index=True)

In [4]:
len(df)

549503

In [5]:
df.columns = ['global_acc1','global_acc2','global_acc3']

In [6]:
t = pd.DataFrame.from_csv("../../data/linear_acceleration_grav_mag.csv")

In [7]:
t.columns

Index(['tester_id', 'TagName', 'Time', 'acc_val1', 'acc_val2', 'acc_val3',
       'acc_val4', 'gravity1', 'gravity2', 'gravity3', 'mag_val1', 'mag_val2',
       'mag_val3'],
      dtype='object')

In [8]:
b = t[['tester_id', 'TagName', 'Time']].copy()

In [9]:
for col in ['global_acc1','global_acc2','global_acc3']:
    b[col] = 0

b[['global_acc1','global_acc2','global_acc3']] = df[['global_acc1','global_acc2','global_acc3']].values
b['unix_timestamp'] = pd.to_datetime(b['Time']).values.astype("uint64") / 1000000000

In [10]:
b.to_csv("../../data/global_acc_df.csv")

In [16]:
from scipy.integrate import simps

In [19]:
features = pd.DataFrame(columns=['unix_timestamp','TagName','tester_id',
                                 'v_1','v_2','v_3','d_1',
                                          'd_2','d_3'])
b_groups = b.groupby(['TagName','tester_id'])
count = 0
for key in b_groups.groups.keys():
    cur_group = b_groups.get_group(key)
    t = cur_group.unix_timestamp.values
    v_1 = []
    d_1 = []
    v_2 = []
    d_2 = []
    v_3 = []
    d_3 = []
    for i in range(len(t)):
        v_1.append(simps(0.8*cur_group.global_acc1.values[:i+1], x=t[:i+1]))
    for i in range(len(t)):
        d_1.append(simps(v_1[:i+1], x=t[:i+1]))
    for i in range(len(t)):
        v_2.append(simps(0.8*cur_group.global_acc2.values[:i+1], x=t[:i+1]))
    for i in range(len(t)):
        d_2.append(simps(v_2[:i+1], x=t[:i+1]))
    for i in range(len(t)):
        v_3.append(simps(0.8*cur_group.global_acc3.values[:i+1], x=t[:i+1]))
    for i in range(len(t)):
        d_3.append(simps(v_3[:i+1], x=t[:i+1]))
    
    v_1 = np.array(v_1)
    d_1 = np.array(d_1)
    v_2 = np.array(v_2)
    d_2 = np.array(d_2)
    v_3 = np.array(v_3)
    d_3 = np.array(d_3)

    temp = np.concatenate((t.reshape(-1,1),np.array([key[0]]*len(t)).reshape(-1,1),np.array([key[1]]*len(t)).reshape(-1,1),d_1.reshape(-1,1),d_2.reshape(-1,1),d_3.reshape(-1,1),v_1.reshape(-1,1),v_2.reshape(-1,1),v_3.reshape(-1,1)), axis = 1)

    features = features.append(pd.DataFrame(temp, columns=['unix_timestamp','TagName','tester_id',
                                                           'v_1','v_2','v_3','d_1',
                                                           'd_2','d_3']), ignore_index=True)
    count+=1
    if (count%50==0):
        print(count)

    

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000


In [20]:
features.to_csv("../../data/global_acc_features_df.csv")

In [19]:
feature['global_acc1'] = b.global_acc1.values
feature['global_acc2'] = b.global_acc2.values
feature['global_acc3'] = b.global_acc3.values
feature['TimeStamp'] = b.Time.values

In [20]:
feature.head()

,unix_timestamp,TagName,tester_id,v_1,v_2,v_3,d_1,d_2,d_3,TimeStamp,global_acc1,global_acc2,global_acc3
0,1.515398e+09,Tag0,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,2018-01-08 15:49:46.379666666+08:00,-0.058561,-0.130082,-0.520824
1,1.515398e+09,Tag0,0.0,-1.834992e-07,-9.703653e-07,-0.000005,-0.000073,-0.000388,-0.002020,2018-01-08 15:49:46.384666666+08:00,0.021859,-0.064001,-0.489335
2,1.515398e+09,Tag0,0.0,-4.303651e-07,-3.528226e-06,-0.000020,0.000035,-0.000564,-0.004001,2018-01-08 15:49:46.389666666+08:00,-0.002340,-0.037200,-0.522459
3,1.515398e+09,Tag0,0.0,-5.876838e-07,-7.142311e-06,-0.000052,-0.000160,-0.000652,-0.006500,2018-01-08 15:49:46.395666666+08:00,-0.070036,0.008021,-0.501289
4,1.515398e+09,Tag0,0.0,-2.218972e-06,-1.039297e-05,-0.000089,-0.000465,-0.000570,-0.008448,2018-01-08 15:49:46.400666666+08:00,-0.069951,0.008737,-0.501289


In [21]:
feature.to_csv("../../data/global_acc_features_df.csv")